In [2]:
import pandas as pd

In [3]:
bandwidth = pd.read_excel('ToyData.xlsx',sheet_name='Inter-Datacenter Links').set_index('Bandwidth/MBps').replace('-',0)
dcd = pd.read_excel('ToyData.xlsx',sheet_name='Data Center Details')
slot = dcd.loc[:,['DC','Num of Slots']].dropna(how='any').set_index('DC')
location = dcd.loc[:,['Data Partition','Location']].set_index('Data Partition')

In [4]:
RQ = pd.read_excel('ToyData.xlsx',sheet_name='Job List').iloc[:,1:].dropna(how='all')
RQ.set_index(list(RQ)[0],inplace=True)
pre = RQ[list(RQ)[-1]].dropna()
exe = RQ[list(RQ)[-2]]
RQ = RQ.iloc[:,0:-2].fillna(0)

In [5]:
pre = pre.apply(lambda x: x.replace("{","").replace("}","")).str.split(", ",expand=True).transpose()

In [6]:
import re

In [7]:
dag = {}
for c in list(pre):
    for l in pre.index.to_list():
        cell = pre[c][l]
        if(cell and cell!= " "):
            edge = re.search(r'\((?P<start>.+),(?P<end>.+)\)',cell,re.S).groupdict()
            if(edge['start'] in dag):
                dag[edge['start']].append(edge['end'])
            else:
                dag[edge['start']] = [edge['end']]

# isolated node
for l in RQ.index.to_list():
    if(l not in dag):
        dag[l] = []

In [8]:
# topological sort
topo_order = []

def dfs(key):
    if(key in topo_order):
        return
    topo_order.append(key)
    if(key in dag):
        for child in dag[key]:
            dfs(child)

for key in dag:
    dfs(key)

# BFS through topological order
stage = {}
visitqueue = list()
for node in topo_order:
    visitqueue.append(node)
    while(len(visitqueue)>0):
        top = visitqueue.pop(0)
        if(top not in stage):
            stage[top] = 0
        if(top in dag):
            for child in dag[top]:
                visitqueue.append(child)
                if((child not in stage) or \
                    (stage[top] + 1 > stage[child])):
                    stage[child] = stage[top] + 1

# group every class of job in every stage
jobs = {}
for tjob in stage:
    jobattr = re.search(r'(\w+)(\d+)',tjob).groups()
    jobclass = jobattr[0]
    jobstage = stage[tjob]
    if(jobclass not in jobs):
        jobs[jobclass] = []
    if(len(jobs[jobclass])<=jobstage):
        jobs[jobclass].append([])
    jobs[jobclass][jobstage].append(tjob)

In [49]:
jobs

{'tB': [['tB1'], ['tB2']],
 'tC': [['tC1'], ['tC2'], ['tC3']],
 'tD': [['tD1', 'tD2'], ['tD3', 'tD4'], ['tD5']],
 'tE': [['tE1'], ['tE2', 'tE3'], ['tE4', 'tE5'], ['tE6']],
 'tF': [['tF1'],
  ['tF2', 'tF3', 'tF4'],
  ['tF5', 'tF6'],
  ['tF7', 'tF8'],
  ['tF9']],
 'tA': [['tA1', 'tA2']]}

In [9]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [66]:
def grouping(suffix,mylist):
    c = {}
    for mem in mylist:
        jobclass = suffix + re.search(r'(\w+)(\d+)',mem).groups()[0]
        if jobclass in c:
            c[jobclass].append(mem)
        else:
            c[jobclass] = [mem]
    return c

In [70]:
dpc = grouping('t', location.index.to_list())
jobclasses = grouping('',RQ.index.to_list())

In [105]:
stagenum = 0
for jobclass in jobs:
    for stage in jobs[jobclass]:
        stagenum = stagenum + 1

# the following solving process is based on 
# gurobi's example on matrix2
m = gp.Model('geodata')

joblist = RQ.index.to_list()
dclist = slot.index.to_list()

# boolean matrix for choosing position of
# the job slot
x = m.addMVar((len(joblist), len(dclist)), vtype=GRB.BINARY, name="schedule")

# the elapsed time of every stage
y = m.addMVar((stagenum), name="time")

# 
jm = m.addMVar((len(joblist)), name="jobmaximum")

# minimize average time
m.setObjective(y.sum()/stagenum,GRB.MINIMIZE)

# subject to:
# (1) job is allocated in row
for i in range(len(joblist)):
    # At most one allocation per row
    m.addConstr(x[i, :].sum() == 1, name="job_"+str(joblist[i]))

# (2) not exceed the number of slot
for j in range(len(dclist)):
    m.addConstr(x[:,j].sum() <= slot["Num of Slots"][dclist[j]], name="dc_"+str(dclist[j]))

# (3) calculate time on every job
stagenum = 0
for jobclass in jobs:
    for stage in jobs[jobclass]:
        # for job in stage:# for prev in jobclasses[jobclass]:
        jobm = [jm[i] for i in range(len(joblist))]
        m.addConstr(y[stagenum]==gp.max_(jobm))
        # m.addConstr(y[stagenum]==gp.max_())
        # m.addConstr(y[stagenum]==gp.max(exe[job] + \
        #     max( \
        #         max(gp.quicksum(x[job][j]*RQ[job][location[k]]/bandwidth[location[k]][j] for j in dclist) for k in dpc[jobclass]), \
        #         max(gp.quicksum(gp.quicksum((x[prev][i] + x[job][j] - abs(x[prev][i] - x[job][j]))/2*RQ[job][prev]/bandwidth[i][j] for j in DC) for i in DC) for prev in jobclasses[jobclass])
        #     ) \
        #     for job in stage \
        #     ),GRB.MAXIMIZE,name=jobclass+':'+str(stagenum))
        stagenum = stagenum + 1

# (4) evaluate max-min


TypeError: bad operand type for unary -: 'GenExpr'